### # 03_purge_and_label.ipynb

In [4]:
# 03_purge_and_label.ipynb

import pandas as pd
import numpy as np
import os


# --- CONFIGURATION ---
DATA_DIR = "../data"
SYMBOL = "EURUSD"
INPUT_FILE = os.path.join(DATA_DIR, f"{SYMBOL}_H1_Engineered.parquet")

# We no longer split files. One giant training set.
OUTPUT_TRAIN = os.path.join(DATA_DIR, "Train_Sniper_Full.parquet")

# --- NEW SNIPER KNOBS (UNSHACKLED) ---

TARGET_HOURS = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] 

# Risk Management: 1:1 Ratio (Stability First)
ATR_PROFIT_FACTOR = 1.0  
ATR_LOSS_FACTOR = 1.0    
LOOKAHEAD_WINDOW = 4     

# --- LOAD DATA ---
print("⏳ Loading engineered data...")
df = pd.read_parquet(INPUT_FILE)

# --- STEP 1: THE SESSION FILTER (EXPANDED) ---
df['hour'] = df.index.hour
session_mask = df['hour'].isin(TARGET_HOURS)
df_sniper = df[session_mask].copy()

print(f"📉 Purged Data: Expanded selection to {len(df_sniper)} candles.")

# --- STEP 2: TRIPLE BARRIER LABELING ---
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=LOOKAHEAD_WINDOW)
df_sniper['fwd_high'] = df_sniper['high'].rolling(window=indexer).max()
df_sniper['fwd_low'] = df_sniper['low'].rolling(window=indexer).min()

# Dynamic Targets
df_sniper['target_up'] = df_sniper['close'] + (df_sniper['ATR'] * ATR_PROFIT_FACTOR)
df_sniper['stop_down'] = df_sniper['close'] - (df_sniper['ATR'] * ATR_LOSS_FACTOR)

df_sniper['target_down'] = df_sniper['close'] - (df_sniper['ATR'] * ATR_PROFIT_FACTOR)
df_sniper['stop_up'] = df_sniper['close'] + (df_sniper['ATR'] * ATR_LOSS_FACTOR)

# --- STEP 3: CLASS ASSIGNMENT ---

# BUY LABEL
df_sniper['LABEL_BUY'] = np.where(
    (df_sniper['fwd_high'] >= df_sniper['target_up']) & 
    (df_sniper['fwd_low'] > df_sniper['stop_down']), 
    1, 0
)

# SELL LABEL
df_sniper['LABEL_SELL'] = np.where(
    (df_sniper['fwd_low'] <= df_sniper['target_down']) & 
    (df_sniper['fwd_high'] < df_sniper['stop_up']), 
    1, 0
)

# Stats
print(f"🎯 Buy Win Rate (Baseline): {df_sniper['LABEL_BUY'].mean():.2%}")
print(f"🎯 Sell Win Rate (Baseline): {df_sniper['LABEL_SELL'].mean():.2%}")

# --- STEP 4: SAVE (SINGLE DATASET) ---
# We keep the Regressor targets just in case, but focused on Classification
df_sniper['REG_TARGET_HIGH'] = np.log((df_sniper['fwd_high'] + 1e-9) / df_sniper['close'])
df_sniper['REG_TARGET_LOW'] = np.log((df_sniper['fwd_low'] + 1e-9) / df_sniper['close'])

# Cleanup
cols_to_drop = ['fwd_high', 'fwd_low', 'target_up', 'stop_down', 'target_down', 'stop_up']
df_sniper.drop(columns=cols_to_drop, inplace=True)

df_sniper.to_parquet(OUTPUT_TRAIN)
print(f"✅ Saved Full Session Data: {len(df_sniper)} rows")

⏳ Loading engineered data...
📉 Purged Data: Expanded selection to 18669 candles.
🎯 Buy Win Rate (Baseline): 33.40%
🎯 Sell Win Rate (Baseline): 32.87%
✅ Saved Full Session Data: 18669 rows
